# 🏗️ Einführung: Strukturierte Outputs & Validierung bei LLMs

Willkommen zu diesem interaktiven Jupyter Notebook!  
Hier lernst du, wie man Large Language Models (LLMs) dazu bringt, strukturierte Ausgaben zu erzeugen – und wie du diese Ergebnisse automatisch validieren kannst.  
Im Fokus steht das leistungsstarke **Gemma 3 4B** Modell von Google, das lokal auf einem Uni-Server läuft. 🖥️

---

## 🧩 Was erwartet dich?

- **Strukturierte Antworten von LLMs:**  
    Erfahre, wie du LLMs so promptest, dass sie klar definierte, maschinenlesbare Formate wie JSON, Listen oder Tabellen ausgeben. Auch Regex-basierte Validierung wird behandelt.

- **Praktische Beispiele:**  
    Wir nutzen das OpenAI-kompatible API-Protokoll, um strukturierte Daten abzufragen und zu überprüfen.
    (Im Backend läuft `python -m vllm.entrypoints.openai.api_server  --trust-remote-code --model google/gemma-3-4b-it --port 27090  --max-model-len 10000 --api-key token-abc123`)
    (Im Backend läuft `python -m vllm.entrypoints.openai.api_server  --trust-remote-code --model Qwen/Qwen3-4B --port 27090  --max-model-len 10000 --api-key token-abc123`)

---

## 📝 Ziel

Am Ende dieses Notebooks wirst du in der Lage sein:

- Prompts zu formulieren, die strukturierte Outputs wie JSON oder Tabellen erzeugen 🗂️
- Die Modellantworten automatisiert zu validieren und Fehler zu erkennen ✅
- Best Practices für robuste, zuverlässige LLM-Workflows zu nutzen 🚦

---

## 📚 Quellen

- [OpenAI: Validating LLM Outputs](https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat)
- [vLLM: Validating LLM Outputs](https://docs.vllm.ai/en/v0.8.2/features/structured_outputs.html

---

Viel Erfolg beim Ausprobieren und Validieren! 🤗

In [5]:
# Importiere die benötigten Bibliotheken
from pydantic import BaseModel, Field
from typing import List, Literal
import requests
import json

In [6]:
# Setze die URL der API, die du ansprechen möchtest
URL = "http://132.199.138.16:27090/v1"

In [7]:
import openai

def execute_llm(prompt, model="google/gemma-3-4b-it", stream=False, seed=0, temperature=0.0, extra_body=None, image_url=None):
    client = openai.OpenAI(
        base_url=URL,
        api_key="token-abc123",
    )

    messages = [
      {
        "role": "user", 
        "content": [
          {
            "type": "text",
            "text": prompt
          }
        ] + ([{"type": "image_url", "image_url": {"url": image_url}}] if image_url else [])
      }
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=stream,
        seed=seed if seed else None,
        extra_body=extra_body if extra_body else None,
    )

    if stream:
        # For streaming, concatenate the content
        return "".join(chunk.choices[0].delta.content or "" for chunk in response)
    else:
        return response.choices[0].message.content


## 1. Guided Choice

"Guided Choice" zwingen das LLM, ausschließlich eine Phrase aus einer vorgegebenen Liste zu wählen.

In [8]:
extra_body = {"guided_choice": ["positive", "negative"]}

# 1. Beispiel Sentimentanalyse
print(execute_llm("Text: Das Essen war super! Sentiment: ", extra_body=extra_body))
print(execute_llm("Text: Das Essen ziemlich schlecht, furchtbar! Sentiment: ", extra_body=extra_body))

positive
negative


In [9]:
# 2. Beispiel Textklassifikation
extra_body = {"guided_choice": ["sports", "politics", "entertainment", "technology"]}
print(execute_llm("Text: Die Olympischen Spiele sind ein großes Ereignis. Kategorie: ", extra_body=extra_body))

sports


In [10]:
# Beispielaufruf: Es ist tatsächlich so, dass unabhängig von der Prompt niemals andere Phrasen als die in extra_body definierten zurückgegeben werden.
extra_body = {"guided_choice": ["Homer", "Marge", "Bart", "Lisa", "Maggie"]}
print(execute_llm("Ein Motor mit 5 Zylindern: ", extra_body=extra_body))

Lisa


## 2. Guided Regex

"Guided Regex" erzwingen, dass die Antwort des LLM einem bestimmten regulären Ausdruck entspricht.

In [11]:
extra_body = {"guided_regex": "\w+@\w+\.com\n"}
# Beispielaufruf: Regex-Validierung
#print(execute_llm("Gib mir die Google URL: ", extra_body=extra_body))

## 3. Guided JSON
"Guided JSON" erzwingen, dass die Antwort des LLM einem bestimmten JSON-Schema entspricht.

In [12]:
# Definiere eine Pydantic-Klasse für einen einzelnen Softwareentwickler
class Developer(BaseModel):
    name: str  # Name des Entwicklers
    programmiersprache: Literal["python", "java", "javascript", "csharp", "cpp", "go", "rust", "php"]  # Hauptprogrammiersprache
    erfahrung_jahre: int = Field(ge=0, le=50)  # Berufserfahrung in Jahren (0-50)
    spezialisierung: Literal["frontend", "backend", "fullstack", "data_science", "devops", "mobile"]  # Spezialisierungsbereich

# Definiere eine Pydantic-Klasse für eine Liste von genau 4 Entwicklern
class DeveloperList(BaseModel):
    entwickler: List[Developer] = Field(min_length=4, max_length=4)  # Genau 4 Developer-Objekte

# Erzeuge das JSON-Schema aus der DeveloperList-Klasse
json_schema = DeveloperList.model_json_schema()

# Konfiguriere extra_body mit dem JSON-Schema für strukturierte Ausgabe
extra_body = {"guided_json": json_schema}

# Führe LLM-Aufruf aus und erzwinge JSON-Struktur
developers = execute_llm("Erstelle eine Liste von 4 Softwareentwicklern: ", extra_body=extra_body)

# Parse den JSON-String in ein Python-Dictionary
developers = json.loads(developers)

# Gib das Ergebnis formatiert und mit deutschen Umlauten aus
print(json.dumps(developers, indent=2, ensure_ascii=False))


{
  "entwickler": [
    {
      "name": "Alice Müller",
      "programmiersprache": "python",
      "erfahrung_jahre": 5,
      "spezialisierung": "data_science"
    },
    {
      "name": "Bob Schmidt",
      "programmiersprache": "java",
      "erfahrung_jahre": 10,
      "spezialisierung": "backend"
    },
    {
      "name": "Clara Weber",
      "programmiersprache": "javascript",
      "erfahrung_jahre": 3,
      "spezialisierung": "frontend"
    },
    {
      "name": "David Klein",
      "programmiersprache": "csharp",
      "erfahrung_jahre": 7,
      "spezialisierung": "mobile"
    }
  ]
}


### Übungsaufgabe zu Guided JSON

Um die Validierung von JSON-Ausgaben zu üben, erstelle ein Prompt, das das LLM anweist, eine strukturierte Antwort im JSON-Format zu liefern.

---

#### 🎯 Aufgabe

Definiere ein einfaches JSON-Schema um Dummy-Daten zu **Musikern** zu generieren.

#### 🎤 Interpret-Schema

| Feld     | Beschreibung                                                |
| -------- | ----------------------------------------------------------- |
| `name`   | Name des Interpreten                                        |
| `age`    | Alter (zwischen 18 und 100)                                 |
| `genre`  | Musikrichtung aus: **Rock**, **Pop**, **Jazz**, **Klassik** |
| `albums` | Liste von Alben                                             |

#### 💿 Album-Schema

| Feld           | Beschreibung                              |
| -------------- | ----------------------------------------- |
| `title`        | Album-Titel                               |
| `release_year` | Erscheinungsjahr (zwischen 1900 und 2023) |
| `tracks`       | Liste von Songtiteln                      |

---

#### 💡 Tipp

Nutze Pydantic-Klassen mit `Field()`-Validierungen und das `Literal`-Typ-System für die Genre-Auswahl!


In [13]:
### Dein Code...

## 4. Guided JSON für strukturierte Extraktion von Informationen aus Dokumenten
Beispiel für strukturierte Extraktion von Informationen aus einem Bild.

Beispiel:

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0s5B5TIgNtd8NBG31BBu2v1cCxIZi3AEE2g&s" alt="Structured Extraction Example" width="100">

In [14]:
# Beginnen wir zunächst ohne eine JSON Validierung, um die Ausgabe zu testen
# Das LLM soll zunächst in der Lage sein, ein Bild zu analysieren und eine Antwort zu generieren

image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0s5B5TIgNtd8NBG31BBu2v1cCxIZi3AEE2g&s"
print(execute_llm("Was siehst du auf diesem Bild: ", image_url=image_url))

Auf dem Bild sehe ich die Mona Lisa, ein berühmtes Gemälde von Leonardo da Vinci. Es ist ein Porträt einer Frau, die lächelt und in die Ferne blickt. Sie trägt ein dunkles Gewand mit bunten Akzenten und sitzt auf einem Stuhl. Im Hintergrund ist eine Landschaft zu sehen. 

Das Gemälde ist bekannt für seine subtile Technik, insbesondere für die "sfumato"-Technik, die einen weichen, verschwommenen Effekt erzeugt und die Konturen der Figur und des Hintergrunds verschmilzt.


### 4. Guided JSON für strukturierte Extraktion von Informationen aus Dokumenten

Nun werden wir ein komplexeres Beispiel betrachten: die strukturierte Extraktion von Informationen aus einem Rechnungsdokument.

#### Ziel der Übung

Wir möchten ein Rechnungsdokument analysieren und die wichtigsten Informationen automatisch extrahieren:
- **Liste der Produkte** mit Details
- **Gesamtsumme** der Rechnung

#### 🖼️ Das Dokument

So sieht das zu analysierende Rechnungsdokument aus:

<img src="https://www.buchhaltungsbutler.de/wp-content/uploads/rechnungsvorlage11-e1692972330525.png" alt="Rechnung Beispiel" width="400">

In [19]:
image_url = "https://www.buchhaltungsbutler.de/wp-content/uploads/rechnungsvorlage11-e1692972330525.png"

# Beginnen wir damit, ein Pydantic-Schema für eine Rechnung zu definieren
class InvoiceItem(BaseModel):
    anzahl: int = Field(ge=0, le=5, description="Anzahl der Artikel")
    einzelpreis: int = Field(le=1000000,description="Preis pro Artikel in Euro")
    beschreibung: str = Field(min_length=1, max_length=100, description="Beschreibung des Artikels")

class Invoice(BaseModel):
    gesamtbetrag: float = Field(gt=0, description="Gesamtbetrag der Rechnung (größer als 0)")
    artikel: List[InvoiceItem] = Field(min_length=1, description="Liste der Artikel (1-5 Artikel)")
    
# Konfiguriere extra_body mit dem JSON-Schema für strukturierte Ausgabe
extra_body = {"guided_json": Invoice.model_json_schema()}
# Führe LLM-Aufruf aus und erzwinge JSON-Struktur
invoice = execute_llm("Analysiere die Rechnung auf dem Bild und gib mir die Details: ", image_url=image_url, extra_body=extra_body)
# Parse den JSON-String in ein Python-Dictionary
invoice = json.loads(invoice)
# Gib das Ergebnis formatiert und mit deutschen Umlauten aus
print(json.dumps(invoice, indent=2, ensure_ascii=False))

{
  "gesamtbetrag": 35,
  "artikel": [
    {
      "anzahl": 5,
      "einzelpreis": 3,
      "beschreibung": "Musterprodukt"
    },
    {
      "anzahl": 3,
      "einzelpreis": 5,
      "beschreibung": "Musterprodukt"
    }
  ]
}
